# NetID:  gg676, xl598, vt152, smk371

# CCA on our data

In [ ]:
import glob
import torch
import os
import pickle
import random
import numpy as np
from sklearn.preprocessing import normalize
from cca_zoo.models import CCA

In [ ]:
random.seed(42)

In [ ]:
linear_cca = CCA(latent_dims=50)

In [ ]:
text_combined_train = torch.load('/common/home/gg676/536/data/text_data/results/combined_features.pth')
text_combined_test = torch.load('/common/home/gg676/536/data/test/text/results/combined_features.pth')

In [ ]:
def load_data(path):
    data_list = []

    for i in glob.glob(path+'/*'):
        x = torch.load(i)
        data_list.extend(x)
    return data_list
img_features_train = load_data('/common/home/gg676/536/data/features/')
img_features_test = torch.load('/common/home/gg676/536/data/test/image/features_images.pth')

In [ ]:
img_features_train = [t.numpy() for t in img_features_train]
text_combined_train = [t.numpy() for t in text_combined_train]
img_features_test = [t.numpy() for t in img_features_test]
text_combined_test = [t.numpy() for t in text_combined_test]

In [ ]:
linear_cca.fit((np.array(img_features_train), np.array(text_combined_train)))

In [ ]:
# Projections of image and text
w1, w2 = linear_cca.transform((np.array(img_features_test), np.array(text_combined_test)))

In [ ]:
im_vecs = w2
instr_vecs = w1
id_data = torch.load('/common/home/gg676/536/data/test/image/id_images.pth')
names_str = id_data

In [ ]:
names = np.array(names_str)
idxs = np.argsort(names)
names = names[idxs]
im_vecs = normalize(im_vecs)[idxs]
instr_vecs = normalize(instr_vecs)[idxs]

In [ ]:
#https://github.com/torralba-lab/im2recipe-Pytorch
N = 10000
embedding = 'image'
idxs = range(N)

glob_rank = []
glob_recall = {1:0.0,5:0.0,10:0.0}
for i in range(10):

    ids = random.sample(range(0,len(names)), N)
    im_sub = im_vecs[ids,:]
    instr_sub = instr_vecs[ids,:]
    ids_sub = names[ids]

    if embedding == 'image':
        sims = np.dot(im_sub,instr_sub.T) # for im2recipe
    else:
        sims = np.dot(instr_sub,im_sub.T) # for recipe2im

    med_rank = []
    recall = {1:0.0,5:0.0,10:0.0}

    for ii in idxs:

        name = ids_sub[ii]
        # get a column of similarities
        sim = sims[ii,:]

        # sort indices in descending order
        sorting = np.argsort(sim)[::-1].tolist()

        # find where the index of the pair sample ended up in the sorting
        pos = sorting.index(ii)

        if (pos+1) == 1:
            recall[1]+=1
        if (pos+1) <=5:
            recall[5]+=1
        if (pos+1)<=10:
            recall[10]+=1

        # store the position
        med_rank.append(pos+1)

    for i in recall.keys():
        recall[i]=recall[i]/N

    med = np.median(med_rank)
    print ("median", med)

    for i in recall.keys():
        glob_recall[i]+=recall[i]
    glob_rank.append(med)

for i in glob_recall.keys():
    glob_recall[i] = glob_recall[i]/10
print ("Mean median", np.average(glob_rank))
print ("Recall", glob_recall)

In [ ]:
from cca_zoo.models import CCA
import torch
import numpy as np
import pickle
import time

In [ ]:
linear_cca = CCA(latent_dims=50)

In [ ]:
def load_pickled_data(path):
    with open(path, 'rb') as fp:
        data = pickle.load(fp)
    return data
text_title_train = load_pickled_data('/common/home/gg676/Downloads/title_embeddings_train.pkl')
text_title_test = load_pickled_data('/common/home/gg676/Downloads/title_embeddings_test.pkl')

In [ ]:
img_text_features_train = load_pickled_data('/common/home/gg676/Downloads/embeddings_train1.pkl')
img_text_features_test = load_pickled_data('/common/home/gg676/Downloads/embeddings_test1.pkl')

In [ ]:
start = time.time()
linear_cca.fit((img_text_features_train[0], text_title_train[0]))
print(time.time() - start)

In [ ]:
# Required for TSNE
def save_projections(file_name, data):
    with open('/common/home/gg676/535/tsne_data/'+file_name, 'wb') as fp:
        pickle.dump(data, fp)
save_projections('image_wrt_title.pkl', w1)
save_projections('title.pkl', w2)

In [ ]:
# Projections of image and text
w1, w2 = linear_cca.transform((img_text_features_test[0], text_title_test[0]))

In [ ]:
# Rest of the code is same as above for calculating the rank